### **Explicación de SelectorGroupChat**

`SelectorGroupChat` es un tipo de equipo en **AgentChat** donde los participantes **se turnan para enviar mensajes a todos los demás miembros del equipo**. A diferencia de `RoundRobinGroupChat`, donde los turnos están predefinidos, en `SelectorGroupChat` **un modelo generativo (como un LLM)** selecciona dinámicamente quién debe hablar en cada momento según el contexto de la conversación.

---

## **📌 Características clave**

1. **Selección de turnos basada en modelo** → Un LLM elige el siguiente agente en función del historial de la conversación.
2. **Roles y descripciones configurables** → Se pueden definir roles específicos para cada agente.
3. **Evita turnos consecutivos del mismo agente** (opcional) → No permite que el mismo agente hable dos veces seguidas (a menos que sea el único disponible).
4. **Personalización de prompts de selección** → Se puede definir cómo el modelo elige el siguiente participante.
5. **Función de selección personalizada (`selector_func`)** → Se puede reemplazar la selección por defecto con una función definida por el usuario.

📢 **Nota:** `SelectorGroupChat` es una **API de alto nivel**. Para mayor personalización, se puede crear una lógica de chat en grupo propia usando la **API Core**.

---

## **🔹 ¿Cómo funciona?**

Cuando el equipo recibe una tarea con `run()` o `run_stream()`, sigue estos pasos:

1. **Análisis del contexto**
    
    - Evalúa el historial de la conversación.
    - Verifica los nombres y descripciones de los agentes.
    - Determina el próximo agente en hablar usando el modelo.
2. **Generación de respuesta**
    
    - El agente seleccionado responde.
    - Su mensaje se envía a todos los miembros del equipo.
3. **Verificación de condición de terminación**
    
    - Si se cumple una condición de terminación (como un límite de mensajes o una palabra clave), la conversación finaliza.
    - Si no, el proceso se repite desde el paso 1.

4. **Devolución del resultado**
    
    - Al finalizar, se retorna un `TaskResult` con el historial de la conversación.


 ![alt text](img/selector-group-chat.svg)

📌 **Importante:**  
El contexto de la conversación **se mantiene** dentro del equipo y los agentes para futuras interacciones. Se puede restablecer con `reset()`.

---

## **🔹 Ejemplo práctico: Búsqueda web y análisis de datos**

En este ejemplo, se crea un equipo de tres agentes especializados:

1️⃣ **Planning Agent** → Coordina la tarea desglosándola en subtareas.  
2️⃣ **Web Search Agent** → Busca información en la web.  
3️⃣ **Data Analyst Agent** → Realiza cálculos con los datos obtenidos.

Se definen herramientas externas:

- `search_web_tool(query: str)`: Simula una búsqueda web.
- `percentage_change_tool(start: float, end: float)`: Calcula el porcentaje de cambio entre dos valores.

### **📍 Creación de los agentes**

Cada agente es una instancia de `AssistantAgent` con su descripción, herramientas y reglas de comportamiento.

In [3]:
# Definir manualmente search_web_tool (simulación de una herramienta de búsqueda web)
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ..."""
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

# Definir manualmente percentage_change_tool (simulación de una herramienta de cálculo)
def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100


In [ ]:
from typing import Sequence
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(model="gpt-4o")

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="Coordina la estrategia y desglosa tareas en subtareas.",
    model_client=model_client,
    system_message="""
    Desglosa tareas complejas en pasos más pequeños.
    No ejecutas tareas, solo las delegas a otros agentes.
    Formato de asignación:
    1. <agente> : <tarea>
    Finaliza con "TERMINATE" cuando todo esté completo.
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="Especialista en búsquedas en la web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="Solo puedes hacer búsquedas web, pero no realizar cálculos.",
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="Analista de datos, útil para cálculos matemáticos.",
    tools=[percentage_change_tool],
    model_client=model_client,
    system_message="Realiza cálculos con las herramientas disponibles.",
)


## **🔹 Ejemplo de ejecución**

Tarea a resolver:

> ¿Quién fue el jugador de los Miami Heat con más puntos en la temporada 2006-2007 y cuál fue el porcentaje de cambio en sus rebotes entre las temporadas 2007-2008 y 2008-2009?

In [ ]:

# 📌 Definir condiciones de terminación
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

# 📌 Crear equipo de agentes
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
)

# 📌 Definir la tarea
task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

# 📌 Ejecutar el equipo
await Console(team.run_stream(task=task))


## **📌 Resumen de SelectorGroupChat**

SelectorGroupChat es una variante de **RoundRobinGroupChat**, pero con una selección dinámica del siguiente agente basado en el contexto. Se puede personalizar con:

- **Selección de turnos basada en un modelo**: un LLM elige qué agente debe hablar según el historial y contexto.
- **Restricción de turnos consecutivos**: evita que el mismo agente hable seguido (aunque esto se puede cambiar con `allow_repeated_speaker=True`).
- **Función personalizada de selección (`selector_func`)**: puedes definir manualmente qué agente debe ser seleccionado en cada paso.
- **Condiciones de terminación**: como `TextMentionTermination("TERMINATE")` o `MaxMessageTermination(25)` para evitar loops infinitos.

---

## **🔍 Observaciones clave sobre la implementación**

1️⃣ **El flujo es correcto**:

- El `PlanningAgent` desglosa la tarea en subtareas y asigna cada una a un agente específico.
- `WebSearchAgent` busca información.
- `DataAnalystAgent` realiza cálculos con los datos obtenidos.
- El proceso se repite hasta que `PlanningAgent` dice "TERMINATE".

2️⃣ **El error con herramientas no disponibles**:

- `DataAnalystAgent` intentó usar `WebSearchAgent` como si fuera una herramienta, lo cual no es correcto.
- Para solucionarlo, asegúrate de que los agentes deleguen las tareas correctamente en lugar de intentar usarse como herramientas.

3️⃣ **Uso de `selector_func`**:

- La función personalizada **fuerza al `PlanningAgent` a hablar después de cada agente**.
- Esto puede ser útil para verificar el progreso antes de continuar con la siguiente tarea.